In [1]:
import csv
csv.field_size_limit(1131072)
import json
import ast

In [2]:
data = []
with open('to_tasa_all_zh_temp.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for item in reader:
        data.append(item)
print(len(data))

## Single Mention Version

In [4]:
# Single Mention Version

output = []
for i, instance in enumerate(data):
    count = 0
    # if i>=5:
    #     continue

    # Modify Target Side Tokens
    src_tokens = ast.literal_eval(instance['src_tokens'])
    tar_tokens = ast.literal_eval(instance['tar_tokens'])
    new_tar_tokens = []
    for k, token in enumerate(tar_tokens):
        new_tar_tokens.append(token)
        new_tar_tokens.append('')

    # Modify Alignments
    alignments = json.loads(instance['config_obj'])['alignment']
    temp = []
    for a in range(len(alignments)):
        for b in range(len(alignments[0])):
            if alignments[a][b]:
                temp.append([a, b])
    new_alignments = []
    for id_0 in range(len(src_tokens)):
        new_alignments.append(
            [False] * len(new_tar_tokens)
        )
    for k, (idx_src, idx_tar) in enumerate(temp):
        new_alignments[idx_src][2*(idx_tar)] = True
        if k!=len(temp)-1:
            new_alignments[idx_src][2*(idx_tar)+1] = True
    src_idns = json.loads(instance['config_obj'])['src_head_inds'][0]

    to_write = {
        'src_tokens': instance['src_tokens'],
        'tar_tokens': json.dumps(new_tar_tokens),
        'config_obj': "{\"src_enable_retokenize\": false, \"version\": {\"PATCH\": 0, \"MAJOR\": 1, \"MINOR\": 0}, \"tar_enable_retokenize\": false,\"src_head_inds\":["+ str(src_idns) +"],\"alignment\":"+str(new_alignments).replace("'","").lower()+"}"
    }
    count += 1
    output.append(to_write)

csv_w = open("to_tasa_zh_drop_test_set.csv", "w", encoding="utf-8")
fieldnames = ['src_tokens', 'tar_tokens', 'config_obj']
writer = csv.DictWriter(csv_w, fieldnames, lineterminator='\n', quoting=csv.QUOTE_ALL)
writer.writeheader()
for line in output:
    writer.writerow(line)

csv_w.close()

## Empty Projection Version

In [17]:
# Empty Single Mention Version

output = []
for i, instance in enumerate(data):
    count = 0
    # if i>=5:
    #     continue

    # Modify Target Side Tokens
    src_tokens = ast.literal_eval(instance['src_tokens'])
    tar_tokens = ast.literal_eval(instance['tar_tokens'])
    new_tar_tokens = []
    for k, token in enumerate(tar_tokens):
        new_tar_tokens.append(token)
        new_tar_tokens.append('')

    # Modify Alignments
    alignments = json.loads(instance['config_obj'])['alignment']
    temp = []
    for a in range(len(alignments)):
        for b in range(len(alignments[0])):
            if alignments[a][b]:
                temp.append([a, b])

    # Only pick empty projection mentions
    if temp:
        continue

    new_alignments = []
    for id_0 in range(len(src_tokens)):
        new_alignments.append(
            [False] * len(new_tar_tokens)
        )
    for k, (idx_src, idx_tar) in enumerate(temp):
        # print(idx_tar, k, len(temp))
        new_alignments[idx_src][2*(idx_tar)] = True
        if k!=len(temp)-1:
            new_alignments[idx_src][2*(idx_tar)+1] = True
    src_idns = json.loads(instance['config_obj'])['src_head_inds'][0]

    to_write = {
        'src_tokens': instance['src_tokens'],
        'tar_tokens': json.dumps(new_tar_tokens),
        'config_obj': "{\"src_enable_retokenize\": false, \"version\": {\"PATCH\": 0, \"MAJOR\": 1, \"MINOR\": 0}, \"tar_enable_retokenize\": false,\"src_head_inds\":["+ str(src_idns) +"],\"alignment\":"+str(new_alignments).replace("'","").lower()+"}"
    }
    count += 1
    output.append(to_write)

csv_w = open("to_tasa_zh_drop_test_set_empty.csv", "w", encoding="utf-8")
fieldnames = ['src_tokens', 'tar_tokens', 'config_obj']
writer = csv.DictWriter(csv_w, fieldnames, lineterminator='\n', quoting=csv.QUOTE_ALL)
writer.writeheader()
for line in output:
    writer.writerow(line)

csv_w.close()

# Multiple Mention Version

In [5]:
# Collect Mentions in the same sentences = []
data_collected = []
current_tokens = []
temp_tasks = []
for i, instance in enumerate(data):
    src_tokens = " ".join(instance['src_tokens'])
    if src_tokens == current_tokens:
        temp_tasks.append(instance)
    else:
        if temp_tasks!=[]:
            data_collected.append(temp_tasks)
        current_tokens = src_tokens
        temp_tasks = [instance]


output = []
for i, sample in enumerate(data_collected):
    instance = sample[0]
    # if i>=100:
    #     continue
    # print(instance['src_tokens'])

    # Modify Target Side Tokens
    src_tokens = ast.literal_eval(instance['src_tokens'])
    tar_tokens = ast.literal_eval(instance['tar_tokens'])
    new_tar_tokens = []
    for k, token in enumerate(tar_tokens):
        new_tar_tokens.append(token)
        new_tar_tokens.append('')

    # Modify Alignments
    temp = []
    src_idn_list = []
    for t_instance in sample:
        alignments = json.loads(t_instance['config_obj'])['alignment']
        for a in range(len(alignments)):
            for b in range(len(alignments[0])):
                if alignments[a][b]:
                    temp.append([a, b])
        src_idn_list.append(str(json.loads(t_instance['config_obj'])['src_head_inds'][0]))

    new_alignments = []
    for id_0 in range(len(src_tokens)):
        new_alignments.append(
            [False] * len(new_tar_tokens)
        )
    for k, (idx_src, idx_tar) in enumerate(temp):
        # print(idx_tar, k, len(temp))
        new_alignments[idx_src][2*(idx_tar)] = True
        if k!=len(temp)-1:
            new_alignments[idx_src][2*(idx_tar)+1] = True
    src_idns = ", ".join(src_idn_list)

    to_write = {
        'src_tokens': instance['src_tokens'],
        'tar_tokens': json.dumps(new_tar_tokens),
        'config_obj': "{\"src_enable_retokenize\": true, \"version\": {\"PATCH\": 0, \"MAJOR\": 1, \"MINOR\": 0}, \"tar_enable_retokenize\": false,\"src_head_inds\":["+ src_idns +"],\"alignment\":"+str(new_alignments).replace("'","").lower()+"}"
    }

    # to_write = {
    #     'src_tokens': instance['src_tokens'],
    #     'tar_tokens': json.dumps(new_tar_tokens),
    #     'config_obj': "{\"src_enable_retokenize\": true, \"version\": {\"PATCH\": 0, \"MAJOR\": 1, \"MINOR\": 0}, \"tar_enable_retokenize\": false,\"src_head_inds\":["+ str(src_idns) + ", "+ str(1) +"],\"alignment\":"+str(new_alignments).replace("'","").lower()+"}"
    # }
    output.append(to_write)
    # print(temp)

csv_w = open("to_tasa_zh_drop_test_set_multiple.csv", "w", encoding="utf-8")
fieldnames = ['src_tokens', 'tar_tokens', 'config_obj']
writer = csv.DictWriter(csv_w, fieldnames, lineterminator='\n', quoting=csv.QUOTE_ALL)
writer.writeheader()
for line in output:
    writer.writerow(line)

csv_w.close()